# SharedPreferences - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu **SharedPreferences** w celu zapisania niewielkich ilości danych w pliku.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExMHZ0Mmt6cjJwYjVrYzE0NWdpbHNqYXM5ZjVvNzc1bmhqZm1zY2F5NSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/kd4nprniMKNt9fBZIv/giphy.gif" width="200" />

`SharedPreferences` to mechanizm używany do przechowywania danych w postaci *klucz-wartość*. Służy do zachowywania małych ilości danych, takich jak ustawienia aplikacji, preferencje użytkownika czy stan sesji. Jest często używany do przechowywania informacji, które nie wymagają zbyt skomplikowanego modelu danych ani relacyjnej bazy danych. Przechowuje dane w plikach `XML`, które są przechowywane wewnątrz katalogu aplikacji.
`SharedPreferences` a `DataStore`:
- Typ danych i bezpieczeństwo:
    - `SharedPreferences` przechowuje dane w plikach `XML` i obsługuje tylko typy prostych danych, takie jak liczby całkowite, ciągi znaków, wartości boolean itp. Jednak nie oferuje wbudowanej obsługi bardziej złożonych struktur danych.
    - `DataStore` obsługuje niestandardowe typy danych i zapewnia automatyczną obsługę konwersji do i z formatu `protobuf`.
- Wsparcie dla asynchroniczności:
    - `SharedPreferences` oferuje operacje synchroniczne, co może wpływać na wydajność aplikacji, szczególnie gdy operacje odczytu/zapisu danych są wykonywane w wątku głównym.
    - `DataStore` został zaprojektowany z myślą o asynchroniczności i wspiera Coroutines, dzięki czemu operacje odczytu/zapisu danych mogą być wykonywane asynchronicznie, co poprawia wydajność i responsywność aplikacji.
- Bezpieczeństwo wątkowe:
    - `SharedPreferences` nie jest wątkowo bezpieczne, co oznacza, że operacje odczytu/zapisu mogą powodować błędy synchronizacji, jeśli są wykonywane równocześnie przez wiele wątków.
    - `DataStore` posiada wbudowane mechanizmy bezpieczeństwa ze względu na wielowątkowość. Można go bezpiecznie używać w aplikacjach wielowątkowych bez konieczności dodatkowej synchronizacji.
- Obsługa zmian danych:
    - `SharedPreferences` nie oferuje wbudowanej obsługi reagowania na zmiany danych. Możemy jedynie odczytać aktualny stan.
    - `DataStore` umożliwia korzystanie z obiektu `Flow` lub `LiveData`, co pozwala na automatyczną obsługę zmian danych. Możemy zarejestrować obserwatora, który będzie otrzymywał powiadomienia o zmianach danych, co ułatwia reagowanie na aktualizacje danych w czasie rzeczywistym.
    
Aplikacja będzie kopią aplikacji z poprzedniego przykładu, dodajmy odpowiednie zależności
```kotlin
    // ViewModel
    implementation ("androidx.lifecycle:lifecycle-viewmodel:2.5.1")
    // LiveData
    implementation ("androidx.lifecycle:lifecycle-livedata:2.5.1")
```

Ponownie wykorzystamy *dummydata* do wygenerowania nazw użytkowników.

In [ ]:
public final class DataProvider {

    private DataProvider(){}

    private static final List<String> usernames = new ArrayList<>();

    static {
        usernames.add("CoolDude123");
        usernames.add("SuperStar");
        usernames.add("GamerGirl99");
        usernames.add("TechMaster");
        usernames.add("MusicLover");
        usernames.add("FitnessFreak");
        usernames.add("Traveler_21");
        usernames.add("FoodieQueen");
        usernames.add("NatureLover");
        usernames.add("Fashionista");
        usernames.add("Bookworm42");
        usernames.add("MovieBuff");
        usernames.add("AdventureSeeker");
        usernames.add("PetLover_87");
        usernames.add("SportsFanatic");
        usernames.add("ArtisticSoul");
        usernames.add("StarGazer");
        usernames.add("YogaEnthusiast");
        usernames.add("PhotographyPro");
        usernames.add("DreamChaser");
        usernames.add("BeachBum123");
        usernames.add("CoffeeAddict");
        usernames.add("GameChanger");
        usernames.add("LaughOutLoud");
        usernames.add("MindfulSoul");
        usernames.add("HappyGoLucky");
        usernames.add("TechGeek1");
        usernames.add("FoodExplorer");
        usernames.add("FitnessJunkie");
    }

    public static String getUsername() {
        Random random = new Random();
        int index = random.nextInt(usernames.size());
        return usernames.get(index);
    }
}

Następnie dodajmy repozytorium, podobnie jak w poprzednich przypadkach, konieczne jest wykorzystanie kontekstu.

In [ ]:
public class UserRepository {

    private final SharedPreferences sharedPref;
    private final MutableLiveData<String> _username = new MutableLiveData<>();

    public UserRepository(Application application) {
        sharedPref = application.getSharedPreferences("fileName", Context.MODE_PRIVATE);
        _username.setValue(sharedPref.getString("username", ""));
    }

    public LiveData<String> getUsername() {
        return _username;
    }

    public void add(String newUsername) {
        SharedPreferences.Editor edit = sharedPref.edit();
        edit.putString("username", newUsername).apply();
        _username.setValue(newUsername);
    }

    public void clear() {
        SharedPreferences.Editor edit = sharedPref.edit();
        edit.putString("username", "").apply();
        _username.setValue("");
    }
}

- `sharedPref = application.getSharedPreferences("fileName", Context.MODE_PRIVATE)` - Wykorzystuje `getSharedPreferences` na obiekcie `Application`, aby uzyskać dostęp do `SharedPreferences` o nazwie `fileName`. Argument `MODE_PRIVATE` wskazuje, że `SharedPreferences` są prywatne dla tej aplikacji i nie są dostępne dla innych aplikacji. Inne tryby:
    - `MODE_APPEND` - pozwala dopisywać kolejne elementy bez nadpisywania
    - `MODE_PRIVATE` - najczęściej wykorzystywany, dostęp do pliku tylko z poziomu aplikacji
    - `MODE_WORLD_READABLE` - zezwala innym aplikacjom na odczyt
    - `MODE_WORLD_WRITABVLE` - zezwala innym aplikacjom na zapis
- `_username.setValue(sharedPref.getString("username", ""))` - Jeśli wartość `username` odczytana jest jako `null`, używamy `""`, aby ustawić domyślną wartość na pusty ciąg znaków.
- `SharedPreferences.Editor edit = sharedPref.edit()` - Tworzy nowy obiekt `SharedPreferences.Editor`, który jest używany do edycji `SharedPreferences`. Dzięki temu edytorowi możemy dokonać zmian, takich jak dodawanie, usuwanie lub aktualizowanie danych.
- `edit.putString("username", newUsername).apply()` 0 W tej linii używamy metody `putString()` na obiekcie edytora, aby dodać nową wartość do `SharedPreferences`. Metoda `putString()` przyjmuje dwa argumenty: klucz (w tym przypadku "username") i wartość (nowa nazwa użytkownika). Wartość zostaje zapisana pod podanym kluczem. Następnie wywołujemy `apply()`, aby zatwierdzić te zmiany.

Pozostałe części aplikacji nie różnią się od poprzedniego przykładu

In [ ]:
class UserViewModelFactory(private val application: Application) :
    ViewModelProvider.Factory {
    override fun <T : ViewModel> create(modelClass: Class<T>): T {
        return UserViewModel(application) as T
    }
}

In [ ]:
public class UserViewModel extends AndroidViewModel {
    private final UserRepository repository = new UserRepository(getApplication());

    public UserViewModel(@NonNull Application application) {
        super(application);
    }

    public LiveData<String> getUsername() {
        return repository.getUsername();
    }

    public void clearUsers() {
        repository.clear();
    }

    public void addUser(String username) {
        repository.add(username);
    }
}


In [ ]:
public class UserFragment extends Fragment {

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        final FragmentUserBinding binding = FragmentUserBinding.inflate(inflater);

        UserViewModel viewModel =new ViewModelProvider(this).get(UserViewModel.class);
        viewModel.getUsername().observe(requireActivity(), binding.textView::setText);

        binding.addButton.setOnClickListener(v -> viewModel.addUser(DataProvider.getUsername()));

        binding.clearButton.setOnClickListener(v -> viewModel.clearUsers());

        return binding.getRoot();
    }
}

Możemy przetestować aplikację.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExMHZ0Mmt6cjJwYjVrYzE0NWdpbHNqYXM5ZjVvNzc1bmhqZm1zY2F5NSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/kd4nprniMKNt9fBZIv/giphy.gif" width="200" />